##### 从头开始实现线性回归

In [ ]:
import random
import torch
from d2l import torch as d2l

In [ ]:
def synthetic_data(w, b, num_samples):
    """生成y = Xw + b噪声"""
    X = torch.normal(0, 1, (num_samples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

d2l.set_figsize()
d2l.plt.scatter(features[:, (1)].detach().numpy(), labels.detach().numpy(), 1)

In [ ]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))  # 生成顺序索引
    random.shuffle(indices)  # 随机打乱索引
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])  # min当num_examples不能被整除时取到
        yield features[batch_indices], labels[batch_indices]  # 根据随机索引取值

batch_size = 10

# for X, y  in data_iter(batch_size, features, labels):
#     print(X, '\n', y)
#     break

In [ ]:
w = torch.normal(0, 0.01, size = (2, 1), requires_grad=True)  # 这个用正态分布随机生成， size即为生成的tensor的大小
b = torch.zeros(1, requires_grad=True)  # requires_grad用于反向传播自动求导

def linreg(X, w, b):
    """线性回归模型"""
    return torch.matmul(X, w) + b  # torch.matmul是torch的tensor乘法

def squared_loss(y_hat, y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad():  # torch.no_grad函数下所有均不会自动求导，覆盖原先required_grad
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [ ]:
lr = 0.03
num_epochs = 10
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

### 线性回归简单实现

In [1]:
import torch
from torch.utils import data
from d2l import torch as d2l
from torch import nn

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [3]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [4]:
net = nn.Sequential(nn.Linear(2, 1))  # 2表示两个参数输入，1表示一个输出
"""
当前net的输出:
    Sequential(
    (0): Linear(in_features=2, out_features=1, bias=True)
    )
"""

net[0].weight.data.normal_(0, 0.01)  # 使用_方法改变初始的w, [0]表示神经网络第一图层
net[0].bias.data.fill_(0)  # 使用_方法改变初始的b, [0]表示神经网络第一图层

loss = nn.MSELoss()  # 均方损失函数

trainer = torch.optim.SGD(net.parameters(), lr=0.03)  # net.parameters()可以自动获取模型中优化的参数

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)  # 通过loss函数求出误差
        trainer.zero_grad()  # 清除之前for循环的求导
        l.backward()  # l在这里会被pytorch发现对应一个函数，会反向传播, 自动求导
        trainer.step()
    l = loss(net(features), labels)  # 计算总误差
    print(f'epoch {epoch + 1}, loss {l:f}')

w = net[0].weight.data
print('w的估计误差:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差:', true_b - b)
